<a href="https://colab.research.google.com/github/0yacinek0/nlp/blob/practice/%5Bgithub_nlp%5D_2_%ED%96%A5%EC%83%81%EB%90%9C_%EB%B6%84%EC%84%9D_%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B0%94%EA%BF%94%EB%B3%B4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MODEL: BI-LSTM

### 구글 드라이브 마운팅 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pwd # 현재 경로 확인 

'/content'

In [4]:
%cd /content/drive/MyDrive/논문/datasets

/content/drive/MyDrive/논문/datasets


In [5]:
pwd

'/content/drive/MyDrive/논문/datasets'

In [6]:
ls

koas_data_test.csv      Mecab-ko-for-Google-Colab/
koas_data_training.csv  tutl-model.pt


### 필요한 패키지와 랜덤시드 설정

In [7]:
# !pip install torchtext==0.10.1 
!pip install torch==1.8.0 torchtext==0.9.0
"""
아마 예전 코드를 실행하시면서 지금은 사라진 객체를 생성하시려다가 생기는 오류 같습니다.

torchtext 버전을 0.10.x 이나 그 이전으로 낮춰보시는 것은 어떠실까요?
pip install torchtext==0.10.1 과 같은 식으로 버전을 지정하셔서 설치하실 수 있습니다.

https://stackoverflow.com/questions/73055161/importerror-cannot-import-name-unicode-csv-reader-from-torchtext-utils

"""

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 735.5/735.5 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 24.1 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1+cu116
    Uninstalling torch-1.13.1+cu116:
      Successfully uninstalled torch-1.13.1+cu116
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.14.1
    Uninstalling torchtext-0.14.1:
      Successfully uninstalled torchtext-0.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.1+cu116 requires torch==1.13.1, but you have torch 1.8.0 which is incompatible.
torchaudio 0.13.1+cu116 requires torch==1.13.1, but you have torch 1.8.0 which is incompatible.


'\n아마 예전 코드를 실행하시면서 지금은 사라진 객체를 생성하시려다가 생기는 오류 같습니다.\n\ntorchtext 버전을 0.10.x 이나 그 이전으로 낮춰보시는 것은 어떠실까요?\npip install torchtext==0.10.1 과 같은 식으로 버전을 지정하셔서 설치하실 수 있습니다.\n\nhttps://stackoverflow.com/questions/73055161/importerror-cannot-import-name-unicode-csv-reader-from-torchtext-utils\n\n'

In [8]:
import torch
from torchtext import data

SEED = 1027

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# 전처리
> Filed 지정 / KoNLPy의 은전한닢(Mecab) tokenizer 이용

In [9]:
# 코랩 내 다운로드 https://wikidocs.net/94600
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

fatal: destination path 'Mecab-ko-for-Google-Colab' already exists and is not an empty directory.
/content/drive/MyDrive/논문/datasets/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 30.4 MB/s eta 0:00:00
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2023-03-07 06:54:23--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 18.205.93.2, 18.205.93.1, 18.205.93.0, ...
Connecting to bitbucket.org (bitbucket.org)|18.205.93.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?respo

In [10]:
from konlpy.tag import Mecab
mecab = Mecab()

In [11]:
import torchtext
from torchtext import data
from torchtext.legacy import data

TEXT = torchtext.legacy.data.Field(tokenize=mecab.morphs, include_lengths = True)
LABEL = torchtext.legacy.data.LabelField(dtype = torch.float)

In [12]:
torch.__version__
torchtext.__version__

'0.9.0'

### torchtext 내 내장된 데이터셋 이용하는게 아니므로 컬럼별 해당 Field 지정

In [14]:
fields = {'sentences': ('text', TEXT), 'Sentiment': ('label', LABEL)}
# dictionary 형식; {csv 컬럼명 : (데이터 컬럼명, Field이름)}

In [15]:
train_data, test_data = data.TabularDataset.splits(
    path = '/content/drive/MyDrive/논문/datasets',
    train = 'koas_data_training.csv',
    test = 'koas_data_test.csv',
    format = 'csv',
    fields = fields
)

In [16]:
vars(train_data[0]), vars(train_data[1])

({'text': ['쯔양',
   '이',
   '댓글',
   '안',
   '달',
   '았',
   '으면',
   '복귀',
   '해도',
   '다',
   '들',
   '뭐',
   '라',
   '안',
   '하',
   '지'],
  'label': '1'},
 {'text': ['완벽',
   '하',
   '게',
   '한글',
   '화',
   '되',
   '어',
   '있',
   '습니다',
   '.',
   '제',
   '가',
   '처음',
   '으로',
   '한',
   '셜록',
   '홈즈',
   '시리즈',
   '이',
   '긴',
   '한데',
   '몰',
   '입도',
   '가',
   '매우',
   '좋',
   '았',
   '습니다',
   '.',
   '다만',
   '좀',
   '뜬금없',
   '는',
   '단서',
   '를',
   '통해',
   '스토리',
   '를',
   '진행',
   '해야',
   '하',
   '는',
   '경우',
   '도',
   '있',
   '으므로',
   '정말',
   '못',
   '찾',
   '겠',
   '다',
   '싶',
   '으면',
   '빨리',
   '구글',
   '검색',
   '을',
   '통해',
   '넘기',
   '는',
   '것',
   '도',
   '방법',
   '입니다',
   '.'],
  'label': '0'})

In [17]:
print(f'훈련 데이터 수: {len(train_data)}')
print(f'테스트 데이터 수: {len(test_data)}')

훈련 데이터 수: 29800
테스트 데이터 수: 8725


### 검증데이터 별도 생성

In [18]:
import random
train_data, valid_data = train_data.split(random_state=random.seed(SEED))

In [19]:
print(f'훈련데이터 수 : {len(train_data)}')
print(f'검증 데이터 수 : {len(valid_data)}')
print(f'테스트 데이터 수 : {len(test_data)}')

훈련데이터 수 : 20860
검증 데이터 수 : 8940
테스트 데이터 수 : 8725


# 단어 벡터는 전처리된 단어 벡터를 받아보기
> * 한글을 지원하는 fasttext.simple.300d 사용 <br>
> * 사전훈련된 단어집에 없는 단어는 0으로 처리하는 걸 방지하기 위해 unk_init = torch.Tensor.normal_옵션을 줌

### 총 단어의 수 확인

In [21]:
MAX_VOCAB_SIZE = 20000

TEXT.build_vocab(train_data,
                 max_size=MAX_VOCAB_SIZE,
                 vectors = 'fasttext.simple.300d',
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [23]:
len(TEXT.vocab)

20002

In [22]:
LABEL.vocab.stoi

defaultdict(None, {'2': 0, '1': 1, '0': 2})

In [24]:
vars(train_data.examples[0])

{'text': ['비타민',
  'D',
  '와',
  '운동',
  '이',
  '면역력',
  '에',
  '중요',
  '한',
  '요소',
  '라는',
  '것',
  '을',
  '알',
  '고',
  '있',
  '었',
  '지만',
  '성별',
  ',',
  '연령',
  '별',
  '로',
  '어떠',
  '한',
  '영향',
  '을',
  '미쳐',
  '면역력',
  '향상',
  '을',
  '위해',
  '평소',
  '영양',
  '상태',
  '를',
  '잘',
  '유지',
  '해야',
  '하',
  '는',
  '동시',
  '주',
  '2',
  '∼',
  '4',
  '회',
  '씩',
  ',',
  '1',
  '∼',
  '2',
  '시간',
  '의',
  '운동',
  '을',
  '꼭',
  '하',
  '는',
  '것',
  '이',
  '좋',
  '다는',
  '것',
  '을',
  '알',
  '게',
  '되',
  '었',
  '습니다',
  '.'],
 'label': '0'}

In [25]:
for i in range(len(train_data)):
  if len(train_data.examples[i].text)==0:
     print(i)

### BuketIterator를 이용하여 데이터 생성자 만들기
* 데이터 생성자를 만드는데, 길이에 따라 정렬하도록 sort_within_batch = True옵션을 넣어줄 것을 원 튜토리얼에서 요구하나
* 한글 데이터에선 오류가 발생하여 sort_key=lambda x:len(x.text) 문장을 먼저 넣어줘야 오류없이 작동함

In [26]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    # sort_within_batch = True,
    sort_key = lambda x: len(x.text),
    sort_within_batch = True,
    device = device
    )

### 패딩 제외 길이로 정렬된 (문장, 길이) 순의 데이터로 이루어져있음

In [27]:
next(iter(train_iterator)).text

(tensor([[11892,   618,   579,    98,    64,  2637,    80,   312,   276,     3,
            638,   314,   801,   913,   444,  1172,   381,   105,   223,   145,
            312,   254,  6362,  1572,  8143,     0,    22,   119,   693,   519,
           3680,   529,  5080,   125,  1095,   222,  1286,  1963,   544,   138,
             22,  6869,   282,   280,    22,   221,  1001,  2162,    49,   151,
            252,   201,  6366,    10, 11747,  2623,   555,  3306,  5130,  4903,
            630,   228,    83,   779],
         [ 3191,    34,    53,    60,   831, 11125,     9,   147,    27,   172,
           5401,    12,  3239,     9,     9,    72,   474,   181,     2,   393,
           1304,   215,   323,  2795,     0,    69,  2157,     7, 11116,    34,
             83,  5443,    18,   190,    12,   547,    18,   278,    11,   174,
           1370,   310, 10817,  1721,    49,  1373,  1057,   214,   126,  7438,
            117,   248,   668,    14,    71,   846,    23,   680,   204,    89,
 

# 모델 생성
* multi-layered bi-directional LSTM 사용
* dropout 적용
* nn.utils.rnn.packed_padded_sequence 써서 패킹/언패킹 처리
<br>
* 최종 hidden size는 [num layers*num directions, batch size, hidden dim]임
* 구체적으로 [forward_layer_0, backward_layer_0, forward_layer_1, backward_layer1, ..., forward_layer_n, backward_layer n] 같이 출력.
* 꼭대기층의 hidden만 필요하므로 hidden[-2::]과 hidden[-1::]만 뽑아 concatenate할 예정

In [28]:
import torch.nn as nn

In [29]:
emb = nn.Embedding(3,5,padding_idx=1) 
test = torch.tensor([0, 1, 2])

In [30]:
emb(test) # padding_idx에 해당하는 벡터는 0임

tensor([[ 0.3671,  0.3455, -0.6553, -0.6129,  0.0402],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 1.5196, -0.0796,  0.8051,  1.1754, -0.2861]],
       grad_fn=<EmbeddingBackward>)

In [31]:
def print_shape(name, data):
  print(f'{name} has shape {data.shape}')

In [32]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim,
                n_layers, bidirectional, dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers,
                          bidirectional=bidirectional, dropout=dropout)
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        # text = [sent_len, batch_size]
        #print_shape('text',text)
        embedded = self.dropout(self.embedding(text))
        # embedded = [sent_len, batch_size, emb_dim]
        #print_shape('embedded', embedded)
        
        # pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        #print_shape('packed_output', packed_output)
        #print_shape('hidden', hidden)
        #print_shape('cell', cell)
        
        # unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        #print_shape('output', output)
        #print_shape('output_lengths', output_lengths)        
        
        # output = [sent_len, batch_size, hi_dim * num_directions]
        # output over padding tokens are zero tensors
        # hidden = [num_layers * num_directions, batch_size, hid_dim]
        # cell = [num_layers * num_directions, batch_size, hid_dim]
        
        # concat the final forward and backward hidden layers
        # and apply dropout
        
        #print_shape('hidden[-2,:,:]', hidden[-2,:,:])
        #print_shape('hidden[-1,:,:]', hidden[-1,:,:])
        #cat = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        #print_shape('cat', cat)

        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        #print_shape('hidden', hidden)
        # hidden = [batch_size, hid_dim * num_directions]
        
        res = self.fc(hidden)
        #print_shape('res', res)
        return res

### 하이퍼파라미터 설정

In [33]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300 # fasttext dim과 동일하게 설정
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

In [34]:
model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
           N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

### 파라미터 갯수 세보기 
* 앞 모델과 비교하여 월등하게 많은 파라미터 수를 확인할 수 있음 

In [35]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'이 모델은 {count_parameters(model):,} 개의 파라미터를 가지고 있다.')

이 모델은 8,720,857 개의 파라미터를 가지고 있다.


### 사전 학습된 fasttxt모델의 단어 벡터를 embedding 레이어에 복사하여 담기

In [36]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([20002, 300])


### Dimension 체크

In [37]:
model.embedding.weight.data.shape

torch.Size([20002, 300])

### weight가 아니라 weight.data 에 덮어씌워야 한다는 걸 명심

In [38]:
model.embedding.weight.data.copy_(pretrained_embeddings) # copy_ 메서드는 인수를 현재 모델의 웨이트에 복사함

tensor([[ 1.1313, -1.2231,  0.7267,  ...,  0.4225, -1.4333,  0.6811],
        [-1.7826,  0.5479,  0.8169,  ..., -0.5085, -0.2250, -1.0241],
        [ 0.0569, -0.0520,  0.2733,  ..., -0.0695, -0.1606, -0.0989],
        ...,
        [-0.5899, -0.0625, -1.1037,  ..., -1.2469, -0.1112,  0.9017],
        [ 0.7496, -0.1232,  0.6312,  ...,  0.6909,  0.0641,  1.4927],
        [ 0.4377,  0.5227,  1.3720,  ...,  1.3790,  0.1214, -1.5727]])

### unk와 pad는 수동으로 0벡터로 만들자.

In [39]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
UNK_IDX, PAD_IDX

(0, 1)

In [40]:
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0569, -0.0520,  0.2733,  ..., -0.0695, -0.1606, -0.0989],
        ...,
        [-0.5899, -0.0625, -1.1037,  ..., -1.2469, -0.1112,  0.9017],
        [ 0.7496, -0.1232,  0.6312,  ...,  0.6909,  0.0641,  1.4927],
        [ 0.4377,  0.5227,  1.3720,  ...,  1.3790,  0.1214, -1.5727]])


### pad는 pad_idx 옵션 때문에 훈련 내내 0으로 남아있지만 unk는 학습될 것!

# 모델 학습
> adam으로 학습 

In [41]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

### 손실 함수는 binary cross entropy with logits로 하자. 이 함수는 임의의 실수를 입력으로 받아서 sigmoid 함수를 취해 0과 1 사이의 값으로 변환한 뒤 label과의 binary cross entropy를 계산

In [42]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

### 평가를 위해 임의의 실수를 0과 1 두 정수 중 하나로 변환하는 함수

In [43]:
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds==y).float()
    acc = correct.sum() / len(correct)
    return acc

### 훈련 함수. 현재 batch.text 는 (토큰들,문장 길이) 로 구성되어 있으니 분리

In [52]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        #print(batch.text)
        text, text_lengths = batch.text
        predictions = model(text, text_lengths).squeeze(1)
        #print_shape('predictions',predictions)
        
        loss = criterion(predictions, batch.label)
        #print_shape('loss',loss)
        
        acc = binary_accuracy(predictions, batch.label)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        print(f"epoch_loss / len(iterator) : {epoch_loss / len(iterator)}")
        print(f"epoch_acc / len(iterator) : {epoch_acc / len(iterator)}")
        
        return epoch_loss / len(iterator), epoch_acc / len(iterator)

### 평가를 위한 함수는 그래디언트 업데이트를 하지 않아야 하므로 with torch.no_grad(): 구문으로 감싸도록 한다. 또한 드랍아웃을 평가 때는 적용하지 않아야 하므로 model.eval() 을 넣어주어야 함

In [53]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            text, text_lengths = batch.text
            predictions = model(text, text_lengths).squeeze(1)

            loss = criterion(predictions, batch.label)#.squeeze(0))
            acc = binary_accuracy(predictions, batch.label)#.squeeze(0))

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
        return epoch_loss / len(iterator), epoch_acc / len(iterator)

### 에폭마다 걸린 훈련시간을 측정하는 함수 

In [50]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

### 훈련 

In [55]:
from tqdm import tqdm 

N_EPOCHS = 10
best_valid_loss = float('inf')

for epoch in tqdm(range(N_EPOCHS)):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
        
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

# 발생한 에러 ValueError: too many values to unpack (expected 2)
# 이 에러의 의미는 설정한 변수의 개수와 리턴해 주는 변수의 개수가 차이가 날 때 발생한다. 
"""
[torchtext.legacy.data.batch.Batch of size 64]
	[.text]:[torch.LongTensor of size 8x64]
	[.label]:[torch.FloatTensor of size 64]
  "batch.text becomes a 2-element tuple when we use include_lengths = True in our TEXT field."
  => If you do not want to use packed padded sequences, then change text, text_lengths = batch.text to text = batch.text
  and remove all references to text_lengths. If you do want to use packed padded sequences, 
  you need include_lengths set to True in your TEXT field
  https://github.com/bentrevett/pytorch-sentiment-analysis/issues/38
  https://bobbyhadz.com/blog/python-valueerror-too-many-values-to-unpack-expected-2
"""

  0%|          | 0/10 [00:00<?, ?it/s]

epoch_loss / len(iterator) : -0.001713441193469463
epoch_acc / len(iterator) : 0.0011023773006134969


 10%|█         | 1/10 [00:24<03:36, 24.07s/it]

Epoch: 01 | Epoch Time: 0m 23s
	Train Loss: -0.002 | Train Acc: 0.11%
	 Val. Loss: -0.568 |  Val. Acc: 38.92%
epoch_loss / len(iterator) : 0.0009915876242280737
epoch_acc / len(iterator) : 0.0015337423312883436


 20%|██        | 2/10 [00:52<03:31, 26.48s/it]

...

 90%|█████████ | 9/10 [03:44<00:24, 24.72s/it]

Epoch: 09 | Epoch Time: 0m 24s
	Train Loss: -0.001 | Train Acc: 0.18%
	 Val. Loss: -1.188 |  Val. Acc: 40.94%
epoch_loss / len(iterator) : -0.006386086253300767
epoch_acc / len(iterator) : 0.0006230828220858896


100%|██████████| 10/10 [04:12<00:00, 25.29s/it]

Epoch: 10 | Epoch Time: 0m 27s
	Train Loss: -0.006 | Train Acc: 0.06%
	 Val. Loss: -1.231 |  Val. Acc: 41.27%


'\n[torchtext.legacy.data.batch.Batch of size 64]\n\t[.text]:[torch.LongTensor of size 8x64]\n\t[.label]:[torch.FloatTensor of size 64]\n  "batch.text becomes a 2-element tuple when we use include_lengths = True in our TEXT field."\n  => If you do not want to use packed padded sequences, then change text, text_lengths = batch.text to text = batch.text\n  and remove all references to text_lengths. If you do want to use packed padded sequences, \n  you need include_lengths set to True in your TEXT field\n  https://github.com/bentrevett/pytorch-sentiment-analysis/issues/38\n  https://bobbyhadz.com/blog/python-valueerror-too-many-values-to-unpack-expected-2\n'

### 테스트셋의 결과

In [56]:
model.load_state_dict(torch.load('tut2-model.pt'))
test_loss, test_acc = evaluate(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: -1.143 | Test Acc: 41.59%


# 사용자 데이터 사용
영화 평가 데이터 직접 넣어보자.

다음 기능을 하는 predict_sentiment 함수를 만들자.

sets the model to evaluation mode
tokenizes the sentence, i.e. splits it from a raw string into a list of tokens
indexes the tokens by converting them into their integer representation from our vocabulary
gets the length of our sequence
converts the indexes, which are a Python list into a PyTorch tensor
add a batch dimension by unsqueezeing
converts the length into a tensor
squashes the output prediction from a real number between 0 and 1 with the sigmoid function
converts the tensor holding a single value into an integer with the item() method

In [58]:
from konlpy.tag import Mecab
mecab = Mecab()

In [59]:
def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok for tok in mecab.morphs(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1) # 배치 
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [60]:
predict_sentiment(model, "이 영화 진짜 재밌었다!!")

0.9999994039535522

In [62]:
predict_sentiment(model, "최악이야!!")

0.33895620703697205

In [63]:
predict_sentiment(model, "괜히 봤음. 내 돈 돌려주세요ㅠ")

0.6688205003738403

In [67]:
predict_sentiment(model, "그냥그랬음")

0.3783482015132904

In [66]:
predict_sentiment(model, "내 인생 영화")

0.9537668228149414